In [ ]:
# Imports

# pandas
import pandas as pd
from pandas import Series,DataFrame
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
# get titanic & test csv files as a DataFrame
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
df = pd.concat([train_df, test_df])
df.reset_index(inplace = True)
df.drop('index', axis = 1, inplace = True)
df = df.reindex_axis(train_df.columns, axis = 1)

In [ ]:
df.shape

In [ ]:
#Feature Engnieering

In [ ]:
import re
df['Title'] = df['Name'].map(lambda x : re.compile(r',(.*?)\.').findall(x)[0].strip())

In [ ]:
df['Title'].value_counts()

In [ ]:
# a map of more aggregated titles
Title_Dictionary = {
                        "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",
                        "Mme":        "Mrs",
                        "Mlle":       "Miss",
                        "Ms":         "Mrs",
                        "Mr" :        "Mr",
                        "Mrs" :       "Mrs",
                        "Miss" :      "Miss",
                        "Master" :    "Master",
                        "Lady" :      "Royalty"
                        }

In [ ]:
df['Title'] = df['Title'].map(Title_Dictionary)

In [ ]:
df['Title'].value_counts()

In [ ]:
#df.head()

In [ ]:
grouped = df.groupby(['Sex','Pclass','Title'])
grouped.median()

In [ ]:
def fillAges(row):
        if row['Sex']=='female' and row['Pclass'] == 1:
            if row['Title'] == 'Miss':
                return 30
            elif row['Title'] == 'Mrs':
                return 45
            elif row['Title'] == 'Officer':
                return 49
            elif row['Title'] == 'Royalty':
                return 39

        elif row['Sex']=='female' and row['Pclass'] == 2:
            if row['Title'] == 'Miss':
                return 20
            elif row['Title'] == 'Mrs':
                return 30

        elif row['Sex']=='female' and row['Pclass'] == 3:
            if row['Title'] == 'Miss':
                return 18
            elif row['Title'] == 'Mrs':
                return 31

        elif row['Sex']=='male' and row['Pclass'] == 1:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 41.5
            elif row['Title'] == 'Officer':
                return 52
            elif row['Title'] == 'Royalty':
                return 40

        elif row['Sex']=='male' and row['Pclass'] == 2:
            if row['Title'] == 'Master':
                return 2
            elif row['Title'] == 'Mr':
                return 30
            elif row['Title'] == 'Officer':
                return 41.5

        elif row['Sex']=='male' and row['Pclass'] == 3:
            if row['Title'] == 'Master':
                return 6
            elif row['Title'] == 'Mr':
                return 26

In [ ]:
df['Age']= df.apply(lambda r : fillAges(r) if np.isnan(r['Age']) else r['Age'], axis=1)

In [ ]:
df.loc[(df['Age']<=13), 'Age_bin_id'] = 0
df.loc[(df['Age']>13) & (df['Age']<=47), 'Age_bin_id'] = 1
df.loc[(df['Age']>47) & (df['Age']<64), 'Age_bin_id'] = 2
df.loc[(df['Age']>=64), 'Age_bin_id'] = 3

In [ ]:
#df['Age'].isnull().sum()

In [ ]:
title_dummies_titanic  = pd.get_dummies(df['Title']).rename(columns = lambda x : 'Title_' + str(x))
df = df.join(title_dummies_titanic)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Name_Len'] = df['Name'].map(lambda s : len(re.split(' ',s)))

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler1 = StandardScaler()
df['Name_len_scaled'] = scaler1.fit_transform(df['Name_Len'])

In [ ]:
df.loc[df['Fare'].isnull(), 'Fare'] = df['Fare'].mean()

In [ ]:
df.loc[df['Embarked'].isnull(), 'Embarked'] = df['Embarked'].dropna().mode().values

In [ ]:
embarked_dummies_titanic  = pd.get_dummies(df['Embarked']).rename(columns = lambda x : 'Embarked_' + str(x))
df = df.join(embarked_dummies_titanic)

In [ ]:
df.loc[df['Cabin'].isnull(), 'Cabin'] = 'U0'
df['CabinLetter'] = df['Cabin'].map(lambda x : re.compile("([a-zA-Z]+)").search(x).group())

In [ ]:
df['CabinLetter'] .value_counts()

In [ ]:
cabin_dummies_titanic  = pd.get_dummies(df['CabinLetter']).rename(columns = lambda x : 'Cabin_' + str(x))
df = df.join(cabin_dummies_titanic)

In [ ]:
pattern = re.compile(r'\D{1}([0-9]+)|(\D{1})')
df['Room'] = df['Cabin'].map( lambda x : pattern.search(x).group(1) if (pattern.search(x).group(1) != None) else 0).astype(int) + 1

In [ ]:
df['Sex'] = df['Sex'].map({'male':1,'female':0})

In [ ]:
pclass_dummies_titanic = pd.get_dummies(df['Pclass'],prefix="Pclass")

In [ ]:
df = df.join(pclass_dummies_titanic)

In [ ]:
def getTicketPrefix(ticket):
    match = re.compile(r'([a-zA-Z./]+)').search(ticket)
    if match :
        return match.group()
    else :
        return 'U'

In [ ]:
def getTicketNumber(ticket):
    match = re.compile(r'([\d]+$)').search(ticket)
    if match:
        return match.group()
    else:
        return '0'

In [ ]:
df['TicketPrefix'] = df['Ticket'].map( lambda x : getTicketPrefix(x.upper()))
df['TicketPrefix'] = df['TicketPrefix'].map( lambda x : re.sub(r'[.?/?]', '', x))
df['TicketPrefix'] = df['TicketPrefix'].map( lambda x : re.sub('STON', 'SOTON', x))

In [ ]:
df.loc[(df['TicketPrefix']=='SOTONOQ') | (df['TicketPrefix']=='SOTONO') |
       (df['TicketPrefix']=='CASOTON'), 'TicketPrefix'] = 'SOTON'

In [ ]:
prefixe_dummies_titanic = pd.get_dummies(df['TicketPrefix'],prefix="Ticket")
df = df.join(prefixe_dummies_titanic)

In [ ]:
df['TicketNumber'] = df['Ticket'].map( lambda x: getTicketNumber(x) )

In [ ]:
df['TicketNumberDigits'] = df['TicketNumber'].map( lambda x: len(x)).astype(np.int)
df['TicketNumberStart'] = df['TicketNumber'].map( lambda x: x[0:1]).astype(np.int)

In [ ]:
df['FamilySize'] = df['Parch'] + df['SibSp'] + 1

In [ ]:
df['Singleton'] = df['FamilySize'].map(lambda s : 1 if s == 1 else 0)
df['SmallFamily'] = df['FamilySize'].map(lambda s : 1 if 2<=s<=4 else 0)
df['LargeFamily'] = df['FamilySize'].map(lambda s : 1 if 5<=s else 0)

In [ ]:
df.loc[(df['Age']<=13), 'Child'] = 1
df.loc[(df['Age']>13), 'Child'] = 0
df.loc[(df['Title_Mrs']==1) & (df['Parch']>0) & (df['Age']>=25), 'Mother'] = 1
df.loc[~((df['Title_Mrs']==1) & (df['Parch']>0) & (df['Age']>=25)), 'Mother'] = 0

In [ ]:
# average survived passengers by Fare_bin
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(10,5))
sur_by_child= df.loc[(df['Survived'].notnull()),["Child","Survived"]].groupby(["Child"],as_index=False).mean()
sur_by_mother= df.loc[(df['Survived'].notnull()),["Mother","Survived"]].groupby(["Mother"],as_index=False).mean()
sns.barplot(x='Child', y='Survived', data=sur_by_child, ax=axis1)
sns.barplot(x='Mother', y='Survived', data=sur_by_mother, ax=axis2)

In [ ]:
df = df.drop(['Name', 'Ticket', 'Cabin', 'Title', 'Name_Len', 'CabinLetter', 'TicketPrefix', 'TicketNumber','Embarked'], axis= 1)

In [ ]:
#df = df.drop(['Embarked'], axis= 1)

In [ ]:
scaler2 = StandardScaler()
df['Age_scaled'] = scaler1.fit_transform(df['Age'])
scaler3 = StandardScaler()
df['Fare_scaled'] = scaler1.fit_transform(df['Fare'])

In [ ]:
df = df.drop(['Age', 'Fare'], axis= 1)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score

In [ ]:
input_df = df.loc[df['Survived'].notnull()]
output_df = df.loc[df['Survived'].isnull()]

In [ ]:
Xtrain = input_df.drop(['Survived','PassengerId'], axis =1)
ytrain = input_df['Survived']
Xtest = output_df.drop(['Survived','PassengerId'], axis =1)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
clf = ExtraTreesClassifier(n_estimators=1000)

In [ ]:
clf = clf.fit(Xtrain, ytrain)

In [ ]:
feature_importance = clf.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
features_list = Xtrain.columns
df_feature = pd.DataFrame({'Feature':features_list, 'Importance':feature_importance}).sort_values('Importance', ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize = (8,12))
feature_plot = df_feature.loc[(df_feature['Importance']>=10),]
sns.set_color_codes("pastel")
sns.barplot(x="Importance", y="Feature", data=feature_plot, label="Importance", color="b")
ax.legend(ncol=1, loc="lower right", frameon=True)
ax.set(xlim=(0,100), ylabel="", xlabel="Importance for Feature greater than 10% of the Max Feature")
sns.despine(left=True, bottom=True)

In [ ]:
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(clf, prefit=True)
Xtrain_new = model.transform(Xtrain)

In [ ]:
Xtrain_new.shape

In [ ]:
Xtest_new = model.transform(Xtest)
Xtest_new.shape

In [ ]:
forest = RandomForestClassifier()

In [ ]:
parameter_grid = {
                 'max_depth' : [5,10,20],
                 'n_estimators': [500, 1000,1500],
                 'max_features': [3, 4, 5]
                 }

In [ ]:
cross_validation = StratifiedKFold(ytrain, n_folds=5)

In [ ]:
grid_search = GridSearchCV(forest,
                           param_grid=parameter_grid,
                           cv=cross_validation, n_jobs=-1)

In [ ]:
grid_search.fit(Xtrain_new, ytrain)

In [ ]:
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

In [ ]:
pred_on_train = grid_search.predict(Xtrain_new)

In [ ]:
ypred = pd.Series(pred_on_train)

In [ ]:
diff = ypred!=ytrain
train_df['Predict_Wrong']=diff
bad_cases = train_df.loc[train_df['Predict_Wrong']==True,:]
dead_but_pred_alive = bad_cases.loc[bad_cases['Survived']==0]
alive_but_pred_dead = bad_cases.loc[bad_cases['Survived']==1]

In [ ]:
dead_but_pred_alive.shape

In [ ]:
alive_but_pred_dead.shape

In [ ]:
from sklearn.learning_curve import learning_curve

In [ ]:
forest = RandomForestClassifier(n_estimators=500, max_features=3, max_depth=10)

In [ ]:
train_sizes, train_scores, val_scores = learning_curve(forest, Xtrain_new, ytrain, cv =10, n_jobs = -1, train_sizes=np.linspace(.1, 1., 10), verbose=0)

In [ ]:
train_scores_mean = np.mean(train_scores, axis = 1)
train_scores_std = np.std(train_scores, axis = 1)
val_scores_mean = np.mean(val_scores, axis = 1)
val_scores_std = np.std(val_scores, axis = 1)

In [ ]:
plt.figure(figsize = (8,8))
plt.title('Random Forest Classifier')
plt.xlabel('Training examples')
plt.ylabel('Score')

plt.gca().invert_yaxis()
plt.grid()

# Plot the std deviation as a transparent range at each training set size
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, 
                 alpha=0.1, color="b")
plt.fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, 
                 alpha=0.1, color="r")

# Plot the average training and test score lines at each training set size
plt.plot(train_sizes, train_scores_mean, 'o-', color="b", label="Train score")
plt.plot(train_sizes, val_scores_mean, 'o-', color="r", label="CV score")

plt.legend(loc = 'best')
# Draw the plot and reset the y-axis
#plt.draw()
#plt.show()

plt.gca().invert_yaxis()

In [ ]:
clf = RandomForestClassifier(n_estimators=1000, max_features=3, max_depth=10)

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
bagging_clf = BaggingRegressor(clf, n_estimators=20, max_samples=0.8, max_features=1.0, bootstrap=True, bootstrap_features=False, n_jobs=-1)

In [ ]:
bagging_clf.fit(Xtrain_new, ytrain)

In [ ]:
output = bagging_clf.predict(Xtest_new).astype(int)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": output_df["PassengerId"],
        "Survived": output
    })
submission.to_csv('output_20161024_1.csv', index=False)